In [3]:
import pandas as pd
import geopandas as gpd
from shapely import geometry
import mercantile
from tqdm import tqdm
import os
import tempfile

## Step 1 - Define our area of interest (AOI)
We define our area of interest (or AOI) as a GeoJSON geometry, then use the shapely library to get the bounding box.

Note: the coordinate reference system for the GeoJSON should be "EPSG:4326", i.e. in global lat/lon format.


In [4]:
# Geometry copied from https://geojson.io
aoi_geom = {
        "coordinates": [
          [
            [
              76.17035694034428,
              10.151854651807724
            ],
            [
              76.17035694034428,
              9.437998857456364
            ],
            [
              76.49615511095192,
              9.437998857456364
            ],
            [
              76.49615511095192,
              10.151854651807724
            ],
            [
              76.17035694034428,
              10.151854651807724
            ]
          ]
        ],
        "type": "Polygon"
      }
aoi_shape = geometry.shape(aoi_geom)
minx, miny, maxx, maxy = aoi_shape.bounds

output_fn = "./geo_data/kochi_microsoft_building.geojson"

## Step 2 - Determine which tiles intersect our AOI

In [5]:
quad_keys = set()
for tile in list(mercantile.tiles(minx, miny, maxx, maxy, zooms=9)):
    quad_keys.add(mercantile.quadkey(tile))
quad_keys = list(quad_keys)
print(f"The input area spans {len(quad_keys)} tiles: {quad_keys}")

The input area spans 2 tiles: ['123321102', '123321120']



## Step 3 - Download the building footprints for each tile that intersects our AOI and crop the results

This is where most of the magic happens. We download all the building footprints for each tile that intersects our AOI, then only keep the footprints that are contained by our AOI.

Note: this step might take awhile depending on how many tiles your AOI covers and how many buildings footprints are in those tiles.


In [6]:
df = pd.read_csv(
    "https://minedbuildings.blob.core.windows.net/global-buildings/dataset-links.csv", dtype=str
)
df.head()

,Location,QuadKey,Url,Size
0,Abyei,122321003,https://minedbuildings.blob.core.windows.net/g...,4.9KB
1,Abyei,122321021,https://minedbuildings.blob.core.windows.net/g...,7.9KB
2,Afghanistan,123011320,https://minedbuildings.blob.core.windows.net/g...,70.1KB
3,Afghanistan,123011321,https://minedbuildings.blob.core.windows.net/g...,1.3MB
4,Afghanistan,123011322,https://minedbuildings.blob.core.windows.net/g...,4.1MB


In [16]:
idx = 0
combined_gdf = gpd.GeoDataFrame()
with tempfile.TemporaryDirectory() as tmpdir:
    # Download the GeoJSON files for each tile that intersects the input geometry
    tmp_fns = []
    for quad_key in tqdm(quad_keys):
        rows = df[df["QuadKey"] == quad_key]
        if rows.shape[0] == 1:
            url = rows.iloc[0]["Url"]

            df2 = pd.read_json(url, lines=True)
            df2["geometry"] = df2["geometry"].apply(geometry.shape)

            gdf = gpd.GeoDataFrame(df2, crs=4326)
            fn = os.path.join(tmpdir, f"{quad_key}.geojson")
            tmp_fns.append(fn)
            if not os.path.exists(fn):
                gdf.to_file(fn, driver="GeoJSON")
        elif rows.shape[0] > 1:
            #raise ValueError(f"Multiple rows found for QuadKey: {quad_key}")
            print(f"Multiple rows found for QuadKey: {quad_key}. Selecting the first one.")
            url = rows.iloc[0]["Url"]  # İlk satırı seçtik
            df2 = pd.read_json(url, lines=True)

            df2["geometry"] = df2["geometry"].apply(geometry.shape)
            gdf = gpd.GeoDataFrame(df2, crs=4326)
            fn = os.path.join(tmpdir, f"{quad_key}.geojson")
            tmp_fns.append(fn)

            if not os.path.exists(fn):
                gdf.to_file(fn, driver="GeoJSON")
        else:
            raise ValueError(f"QuadKey not found in dataset: {quad_key}")

    # Merge the GeoJSON files into a single file
    for fn in tmp_fns:
        gdf = gpd.read_file(fn)  # Read each file into a GeoDataFrame
        #gdf = gdf[gdf.geometry.within(aoi_shape)]  # Filter geometries within the AOI
        gdf['id'] = range(idx, idx + len(gdf))  # Update 'id' based on idx
        idx += len(gdf)
        combined_gdf = pd.concat([combined_gdf,gdf],ignore_index=True)

  0%|                                                                                                                                                                                                                                                     | 0/2 [00:00<?, ?it/s]

Multiple rows found for QuadKey: 123321102. Selecting the first one.


 50%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                                                                                                      | 1/2 [00:00<00:00,  1.39it/s]

Multiple rows found for QuadKey: 123321120. Selecting the first one.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.24it/s]


In [17]:
len(combined_gdf)

634


## Step 4 - Save the resulting footprints to file


In [18]:
combined_gdf = combined_gdf.to_crs('EPSG:4326')
combined_gdf.to_file(output_fn, driver='GeoJSON')